Recurrent Neural Network model for regression in sequences
====

In this example we will use the RNNModel to set up an experiment over one of the Spice (http://spice.lif.univ-mrs.fr/index.php) competence for sequence prediction, held in 2016. We will start by downloading and preprocessing the dataset.

In [1]:
ORIGIN_URL = 'http://spice.lif.univ-mrs.fr/data/1.spice.train'
DATASET_DIR = 'downloads'
DATASET_FILENAME = 'spice_dataset.txt'

In [2]:
# add parent directory to python path
import sys
sys.path.append('../')

In [3]:
import numpy
import os
import urllib

In [4]:
import utils
utils.safe_mkdir(DATASET_DIR)

In [5]:
def maybe_download():
    """Downloads dataset if it doesn't exists"""
    filename = os.path.join(DATASET_DIR, DATASET_FILENAME)
    if os.path.exists(filename):
        return
    urllib.urlretrieve(ORIGIN_URL, filename)

maybe_download()

The dataset file consists on a series of numerical sequences, one per line, including a header line that we will ignore. We will try to predict the last element of each sequence. 

In [6]:
def read_dataset():
    """Reads the dataset. Returns a list with sequences and a list of labels"""
    with open(os.path.join(DATASET_DIR, DATASET_FILENAME), 'r') as input_file:
        lines = input_file.readlines()[1:]  # Ignore the header
    # Split lines and convert numbers to int.
    sequences = [[numpy.array([int(value)], dtype=numpy.int16) for value in line.split()] for line in lines]
    instances = [sequence[:-1] for sequence in sequences]
    labels = [sequence[-1] for sequence in sequences]
    return numpy.array(instances), numpy.array(labels)

In [7]:
instances, labels = read_dataset()

We can now create the dataset using the extracted instances and labels

In [28]:
import dataset
dataset = reload(dataset)

samples = 4
partition_sizes = {'train': 0.7, 'test': 0.2, 'validation': 0.1}

splice_dataset = dataset.SequenceDataset()
splice_dataset.create_samples(instances, labels, samples, partition_sizes, use_numeric_labels=True)

In [39]:
import experiment
experiment = reload(experiment)
from models import lstm
lstm = reload(lstm)

config = {
    'model': lstm.LSTMModel,
    'model_arguments': {'hidden_layer_size': [50], 'batch_size': 100,
                        'logs_dirname': '../../results/examples/splice/',
                        'log_values': True, 'training_epochs': 100}
}
splice_experiment = experiment.SampledExperiment(splice_dataset, config=config)


In [40]:
splice_experiment.run()

Tensor("sequences_placeholder:0", shape=(?, 30, 1), dtype=int16)


AttributeError: 'list' object has no attribute 'get_shape'